# Unlabeled histories of small trees with the uniform model

In this code I will make some computations. My goal is to create a dataset of all unlabeled histories of small size (maybe $n\leq 9$ to start)

In [2]:
import copy
# let's create a class to help us with our unlabelled histories

class UnlabeledHistory:

    # constructor (by default creates a tree after one split)
    def __init__(self, A = [0], omega = [0]):
        
        self.A = A
        self.omega = omega
        self.t = len(A)
        self.sackin = 2*(self.t) + sum(self.omega)
        
        counts = [0]*(self.t + 1)
        for num in self.A:
            counts[num]+=1
        self.counts = counts
        
        self.adj = [i for i in range(1, self.t+1) if self.counts[i] < 2]
        
    # returns an array where the value of index i is how many times i appeared in A
    def get_counts(self):
        return(self.counts)
    
    # methods to obtain values of our class
    def get_A(self):
        return(self.A)
    
    def get_omega(self):
        return(self.omega)
    
    def get_t(self):
        return(self.t)
    
    def get_sackin(self):
        return(self.sackin)

    def get_adj(self):
        return(self.adj)
    
    # method which appends an element a to a current unlabeled history
    def app(self, a):
        if a not in self.adj:
            return("Not a valid leaf")
        self.A.append(a)
        new_O = self.omega[a-1]+1
        self.omega.append(new_O)
        self.t += 1
        self.sackin += 2 + new_O
        self.counts.append(0)
        self.counts[a] += 1
        
        if (self.counts[a] >= 2):
            self.adj.remove(a)
        self.adj.append(self.t)
        return(self)
    
    # method which returns a new unlabeled history consisting of the current unlabeled history with label a created
    def add(self,a):
        new_history = copy.deepcopy(self)
        new_history.app(a)
        return(new_history)
    
    # method which returns a list of all of the unlabeled histories which can be created by appending one element
    # to the current unlabeled history
    def adjacent(self):
        adjacent = []
        for a in self.get_adj():
            adjacent.append(self.add(a))
        return(adjacent)
    
    
    # enables us to export an unlabeled history as a dictionary to make it easy to create a dataframe out of it
    def to_dict(self):
        return {'A': self.A, 'Omega': self.omega, 't' : self.t, 'Sackin': self.sackin, 'Counts' : self.counts, 'Adj' : self.adj}
           
    
        
        
        

In [8]:

# now let's get to work with everything! 

# I will create a list which contains all of the UnlabelledHistory's which are possible up to size t = n

n = 11 # for n = 10, it takes about 3 seconds on my machine. n = 11 around 10
all_histories = []
first = UnlabeledHistory()

all_histories.append([first])
for i in range(n-1):
    new_histories = []
    for history in all_histories[i]:
        adj = history.adjacent()
        for a in adj:
            new_histories.append(a)
    all_histories.append(new_histories)
    
for i in range(n):
    print(len(all_histories[i]))
    

1
1
2
5
16
61
272
1385
7936
50521
353792


In [9]:
# now, we can make a dataframe of all of the data to do some data exploration
import pandas as pd

data = [x.to_dict() for j in range(n) for x in all_histories[j] ]

df = pd.DataFrame(data)
df.head(20)

,A,Omega,t,Sackin,Counts,Adj
0,[0],[0],1,2,"[1, 0]",[1]
1,"[0, 1]","[0, 1]",2,5,"[1, 1, 0]","[1, 2]"
2,"[0, 1, 1]","[0, 1, 1]",3,8,"[1, 2, 0, 0]","[2, 3]"
3,"[0, 1, 2]","[0, 1, 2]",3,9,"[1, 1, 1, 0]","[1, 2, 3]"
4,"[0, 1, 1, 2]","[0, 1, 1, 2]",4,12,"[1, 2, 1, 0, 0]","[2, 3, 4]"
5,"[0, 1, 1, 3]","[0, 1, 1, 2]",4,12,"[1, 2, 0, 1, 0]","[2, 3, 4]"
6,"[0, 1, 2, 1]","[0, 1, 2, 1]",4,12,"[1, 2, 1, 0, 0]","[2, 3, 4]"
7,"[0, 1, 2, 2]","[0, 1, 2, 2]",4,13,"[1, 1, 2, 0, 0]","[1, 3, 4]"
8,"[0, 1, 2, 3]","[0, 1, 2, 3]",4,14,"[1, 1, 1, 1, 0]","[1, 2, 3, 4]"
9,"[0, 1, 1, 2, 2]","[0, 1, 1, 2, 2]",5,16,"[1, 2, 2, 0, 0, 0]","[3, 4, 5]"


In [10]:
stats = df.groupby('t')['Sackin'].agg(['mean', 'var']).reset_index()

# Rename the columns for clarity
stats.columns = ['t', 'Sackin Mean', 'Sackin Variance']

print(stats)

     t  Sackin Mean  Sackin Variance
0    1     2.000000              NaN
1    2     5.000000              NaN
2    3     8.500000         0.500000
3    4    12.600000         0.800000
4    5    17.062500         1.529167
5    6    21.885246         2.669945
6    7    26.988971         4.357811
7    8    32.348736         6.610231
8    9    37.931074         9.468342
9   10    43.715109        12.950447
10  11    49.681624        17.079387


In [11]:
# let's compare to what we expected according to the book

import math
# assume n >=1 
def doubleFactorial(n):
    if n <= 1:
        return 1
    else:
        return n * doubleFactorial(n-2)
    
def expectedMean(t):
    ex = (t+1)*(doubleFactorial(2*t)/doubleFactorial(2*t - 1) - 1)
    return ex

def expectedVariance(t):
    n = t+1
    var = n*(10*n**2 - 3*n - 1)/3  \
        - math.comb(n+1, 2)*doubleFactorial(2*t)/doubleFactorial(2*t - 1) \
        - n**2 * (doubleFactorial(2*t)/doubleFactorial(2*t - 1))**2
    return var


stats['Expected Mean'] = stats['t'].apply(expectedMean)
stats['Expected Variance'] = stats['t'].apply(expectedVariance)

print(stats)

     t  Sackin Mean  Sackin Variance  Expected Mean  Expected Variance
0    1     2.000000              NaN       2.000000           0.000000
1    2     5.000000              NaN       5.000000           0.000000
2    3     8.500000         0.500000       8.800000           0.160000
3    4    12.600000         0.800000      13.285714           0.775510
4    5    17.062500         1.529167      18.380952           2.235828
5    6    21.885246         2.669945      24.030303           4.999082
6    7    26.988971         4.357811      30.191142           9.576518
7    8    32.348736         6.610231      36.829371          16.521935
8    9    37.931074         9.468342      43.916907          26.424194
9   10    43.715109        12.950447      51.430102          39.901699
10  11    49.681624        17.079387      59.348688          57.598180
